# Modelação Molecular 2023/24
## TP2: Modelação de uma Reacção Química: Parte I

A Substituição nucleófilica bimolecular (SN2) é uma das reacções mais importantes em química orgânica, não só pelas possibilidades sintéticas que abre, mas também enquanto passo elementar de várias reacções mais complexas.

Neste trabalho estudar-se-á uma série de reacções SN2 com diferentes nucleófilos e grupos de saída usando métodos de Mecânica Quântica. Este trabalho encontra-se dividido em duas partes. Na primeira parte será realizado o estudo da SN2 usando um método rápido (xTB). A energia correspondente aos estados estacionários de cada reacção (reagentes, produtos e estados de transição) será refinada usando cálculos DFT, os quais irão correr num servidor no período entre aulas práticas. Na segunda parte do trabalho serão compilados e interpretados os resultados deses cálculos. 

Este trabalho poderá correr tanto nos computadores da Universidade do Minho.

In [ ]:
platform='UMINHO' # ou 'COLLAB'

In [ ]:
# preparação do ambiente python
import subprocess
import numpy as np
import matplotlib.pyplot as plt

atomic_symbols=['XX','H','He','Li','Be','B','C','N','O','F','Ne',
                'Na','Mg','Al','Si','P','S','Cl','Ar'
                'K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Ge','As','Se','Br','Kr',
                'Rb','Sr','Y','Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Te','I','Xe'                
               ]

def shell(cmd):
    p=subprocess.Popen(cmd,stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE,shell=True)
    output, error = p.communicate()
    return output,error

def plot_trj_energy(fn):
  data=open(fn,'r').readlines()
  e=list()
  for line in data:
    if 'energy:' in line:
      l=line.split()
      for i,token in enumerate(l):
        if token=='energy:':
          e.append(float(l[i+1]))
  e = np.array(e)
  e = 2625.5*(e-e.min())
  fig,ax = plt.subplots()
  ax.scatter(np.arange(1,len(e)+1),e)
  ax.set_xlabel("Passo (n)")
  ax.set_ylabel("Energia Relativa (kJ/mol)")
  plt.show()

def extract_geo(ifn, n, ofn):
  """Extrai a geometria n de um ficheiro do tipo xtb*.log (começando em 1) e 
  guarda essa geometria num ficheiro chamod ofn"""
  with open(ifn,'r') as f:
    data=f.readlines()
  natoms = int(data[0])
  start = (natoms+2)*(n-1)
  end = start + natoms + 2
  with open(ofn,'w') as f:
    for line in data[start:end]:
      f.write(line)

def g98_animate_mode(fn,n_vib,ofn,scale=1.0):
    data = open(fn,'r').readlines()
    ZZ = list()
    geo = list()
    displ = list()
    # ler a geometria
    geo_start=100000000000000
    geo_end = -1
    for n,line in enumerate(data):
        if 'Coordinates' in line:
            geo_start = n+3
        if (n>geo_start) and '--------' in line:
            geo_end = n
            break
    for n in range(geo_start,geo_end):
        l =  data[n].split()
        ZZ.append(int(l[1]))
        geo.append(list(map(float,l[-3:])))
    geo=np.array(geo)
    symbols = [atomic_symbols[z] for z in ZZ]
    natoms = len(symbols)
    vib_start = geo_end + 7
    vib_step = 9 + natoms
    for i in range(vib_start,len(data),vib_step):
        if n_vib in list(map(int,data[i].split())):
            v_ind = list(map(int,data[i].split())).index(n_vib)
            v_start = i + 9
            v_end = i + vib_step
            for j in range(v_start,v_end):
                l = data[j].split()
                displ.append(list(map(float,l[2+(3*v_ind):5+(3*v_ind)])))
            break
    displ=np.array(displ)*scale
    delta = np.sin(np.linspace(0,6.28,20))
    xyz=''
    for dl in delta:
        this_geo=geo+(dl*displ)
        xyz+=f"{natoms}\n* (null), Energy   -1000.0000000\n"
        for n in range(natoms):
            xyz += f"{symbols[n]:3s}"
            for j in range(3):
                xyz += f" {this_geo[n][j]:10.6f}"
            #for j in range(3):
            #    xyz += f" {displ[n][j]}"
            xyz += '\n'
    print(xyz,file=open(ofn,'w'))  

if platform.upper()=='HOME':
    def run_xtb(args):
        return shell(f"ulimit -s unlimited; OMP_STACKSIZE=12G; OMP_MAX_ACTIVE_LEVELS=1 xtb {args}")[0].decode()
    
if platform.upper()=='UMINHO':
    def run_xtb(args):
        return shell(r'C:\Users\Public\Documents\xtb-6.6.1-windows-x86_64\xtb-6.6.1\bin\xtb '+f"{args}")[0].decode()

if platform.upper()=='COLLAB':
    !wget -c https://github.com/grimme-lab/xtb/releases/download/v6.5.1/xtb-6.5.1-linux-x86_64.tar.xz
    !tar xf xtb-6.5.1-linux-x86_64.tar.xz
    !export PATH=/content/xtb-6.5.1/bin:${PATH}
    def run_xtb(args):
        return shell(f"ulimit -s unlimited; OMP_STACKSIZE=12G; OMP_MAX_ACTIVE_LEVELS=1 /content/xtb-6.5.1/bin/xtb {args}")[0].decode()

## Sistemas a estudar

Cada aluno deve optar por uma das seguintes opções:

#### Opção I: estudo do efeito do grupo de saída
Nesta opção será estudada a SN2 entre a etilamina e os seguintes compostos: fluorometano, clorometano, bromometano e iodometano. Todas as reacções serão estudadas em diclorometano.

#### Opção II: estudo da influência do solvente
Nesta opção será estuda a SN2 entre a etilamina e o bromometano, nos seguintes solventes: diclorometano, hexano, acetonitrilo e água.

Em todos os casos, serão calculados dados termodinâmicos (variação de energia, entalpia e energia de Gibbs) e cinéticos (energia de activação e energia de Gibbs de activação) a 298 K.

A reacção SN2 entre a metilamina e o bromometano em diclorometano é comum a ambas as opções, pelo que será usada como tutorial.

## Procedimento Geral

O estudo do perfil de uma reacção elementar começa pela optimização da geometria do sistema num dos estados iniciais (reagentes) ou finais (produtos): podemos chamar esse estado genericamante como o nosso estado de partida. Isto é feito com um cálculo do tipo optimização de geoemtria. A observação da geoemtria de equílibrio no estado de partida permite deteminar duas coisas:
* As aproximações ou os afastametnos entre átomos que devem ocorrer para que a reacção se dê, e
* Os constrangimentos que poderão ser aplicados ao sistema de forma a evitar seguir por caminhos alternativos

Após essa análise, preparam-se um ou mais cálculos nos quais as distâncias entre os átomos de interesse são fixadas em valores determinados. Chamamos a este tipo de cálculos de varrimento (_scan_). No final destes cálculos podemos usar um programa de visualização para verificar se o scan for bem sucedido em ligar o ponto de partida ao ponto de chegada desejado. Também deverá ser possível identificar um máximo da energia ao longo do caminho que corresponde ao **Estado de Transição** que governa esta reacção.

Normalmente, a geometria do estado de trnasição é optimizada de forma a minimizar a energia em relação a todos os outros movimentos atómicos que não façam parte da coordenada de reacção. Em certos casos, um _scan_ suficientemente fino e com critérios de optimização apertados pode ser suficiente para se obter uma boa aproximação ao estado de transição (o que será feito neste trabalho). Um estado de transição deve ter as seguintes características:
* **Geometria próxima do equilíbrio** isto pode ser verificado por um valor baixo da norma do gradiente (GRADIENT NORM), e
* **Uma (e apenas uma) frequência vibracional negativa ou imaginária** Esta "vibração" imaginária corresponde ao movimento ao longo da coordenada de reacção.
Se distorcermos ligeiramente o estado de transição para um e outro lado da coordenada de reacção, e optimizarmos cada uma das geometrias distorcidas, deveremos chegar aos reagentes e produtos da reacção estudada, _na sua conformação mais próxima à do caminho reaccional_. Este procediemnto é uma de várias ferramentas usadas para garantir que o estado de transição efectivamente medeia o processo em estudo.

Finalmente, devem-se recolher os valores absolutos da energia, entalpia e função de Gibbs de cada um dos três estados (reagentes, prodtos e estado de transição) de forma a poderem ser calculadas as respectivas variações.

## Reacção SN2 entre a etilamina e o bromometano em diclorometano

In [ ]:
# o seguinte codigo python escreve um ficherio XYZ com o sistema na sua configuração inicial (reagentes)
# depois de correr esta célula pode (deve!) usar o vmd ou jmol para visualizar o sistema e identificar os átomos que vão participar na reacção.

xyz="""   15
 Sn2 template
 C    -1.786579     1.250095    -0.026275
 C    -0.481434     0.560416    -0.424879
 N     0.186750    -0.148934     0.659731
 H    -1.602525     2.006507     0.734340
 H    -2.494881     0.524788     0.370369
 H    -2.233935     1.731766    -0.891000
 H     0.221190     1.302229    -0.813197
 H    -0.679704    -0.165200    -1.217936
 H     0.383415     0.500091     1.415768
 H    -0.439078    -0.855871     1.034188
 C     2.275898    -1.169264    -0.063920
 Br    4.166021    -2.092157    -0.718571
 H     2.087334    -1.559952     0.920787
 H     2.454211    -0.108394    -0.070302
 H     1.540423    -1.475307    -0.787118
"""

open("etilamina-mebr-dcm-react-guess.xyz",'w').write(xyz)

Num trabalho de Modelação Molecular é importante manter os ficheiros organizados. Neste trabalho, todos os ficheiros contém a seguinte informação, separada por hifens (e **sempre sem espaços, acentos ou cedilhas!**): nucleofilo-electrofilo-solvente-estado-outra_informação_relevante. Os cálculos xTB serão levados a cabo por um programa chamado ```xtb```, o qual corre normalmente na linha de comandos e usa argumentos e opções para controlar o seu comportamento. Para correr o xtb, foi definida acima uma função do python que trata das diferenças entre plataformas, sendo necessário apenas fornecer uma cadeia de carateres (_string_) com as opções e argumetnos necessários a cada cálculo. 

In [ ]:
# exemplo para correr o xtb
# calcula a energia electronia do sistema inicial
# o output do xtb 
print(run_xtb("etilamina-mebr-dcm-react-guess.xyz --scc"))

### Optimização da geometria do estado de partida
O cálculo abaixo é semelhante ao anterior, mas é pedida uma optimização da geometria (```--opt```), indicamos o solvente (```--açpb```) também definimos um nome comum para todos os ficheiros que o xtb produz (```--namespace```, neste caso, todos os ficheiros irão começar por etilamina-mebr-dcm-react-opt). Para além disso, também usamos a opção ```file``` da função ```print()``` para colocar o output do xtb num ficheiro de texto, em vez de aparece no notebook.

In [ ]:
print(run_xtb("etilamina-mebr-dcm-react-guess.xyz --opt --namespace etilamina-mebr-dcm-react-opt --alpb CH2Cl2"), file=open('etilamina-mebr-dcm-react-opt.out','w'))

Dos ficheiros criados neste cálculo, destacam-se os seguintes ficheiros:
* ```etilamina-mebr-dcm-react-opt.out```: contém o output do xtb
* ```etilamina-mebr-dcm-react-opt.xtbopt.xyz```: contém a geometria de equílibrio, necessário para cálculos futuros.
* ```etilamina-mebr-dcm-react-opt.xtbopt.log```: contém as geometrias (e respectivas energias) pelas quais o sistema passou desde a geometria inicial até à geometria de equilíbrio. Para visualizar este ficheiro no VMD, é nessário indicar que se trata de um ficheiro XYZ na caixa de texto "Determine File Type" da janela "Molecule File Browser" onde carregamos os ficheiros. Podemos também ver como varia a energia do sistema ao longo da optimização com a função ```plot_trj_energy()``` definida acima.

In [ ]:
plot_trj_energy('etilamina-mebr-dcm-react-opt.xtbopt.log')

### Scan ao longo do caminho reacional
Da análise deste sistema, verificamos que para a reacção SN2 ter lugar, é necessário aproximar os átomos N3 e C11 (no VMD, o indice dos átomos começa em 0, pelo que é necessário somar 1 ao resultado de uma Query). Também é necessário manter o átomo de Br orientado de forma a que o ângullo N2-C11-Br seja próximo de 180º. 

Precisamos então de configurar o xtb para que faça um varrimento da aproximação destes átomos, mantendo a orientação relativa do Br. Isto é feito num ficheiro de configraçao que pode ser reutilizados para outros cálculos semelhantes.

In [ ]:
config="""$cma

$constrain
 force constant=6.0
 distance: 3, 11, auto
 angle: 3, 11, 12, 180
$end

$scan
 1: 4.0, 1.2, 200
end

"""

print(config,file=open('scan.cfg','w'))

print(run_xtb("etilamina-mebr-dcm-react-opt.xtbopt.xyz --opt --input scan.cfg --namespace etilamina-mebr-dcm-react-scan --alpb CH2Cl2"), file=open('etilamina-mebr-dcm-react-scan.out','w'))

Poderá agora verificar a trajectória ao longo do varrimento (```etilamina-mebr-dcm-react-scan.xtbscan.log```) e fazer o gráfico da variação de energia ao longo da mesma

In [ ]:
plot_trj_energy('etilamina-mebr-dcm-react-scan.xtbscan.log')

O ponto 137 parece ser o máximo da energia ao lonog da trajectória (descondando os últimos pontos, nos quais a compressão da ligação CN está a aumentar a energia do sistema. Devemos então extrair o mesmo da trajectória e realizar a análise de frequências.

In [ ]:
extract_geo('etilamina-mebr-dcm-react-scan.xtbscan.log',137,'etilamina-mebr-dcm-ts-geo.xyz')

In [ ]:
print(run_xtb("etilamina-mebr-dcm-ts-geo.xyz --hess --namespace etilamina-mebr-dcm-ts-freq --alpb CH2Cl2"), file=open('etilamina-mebr-dcm-ts-freq.out','w'))

Embora o xtb avise sobre a possibilidade de não estarmos numa geometria de equilibrio, a norma do gradiente é sufcientemente pequena para este trabalho. Os cálculos de freqências geram alguns ficheiros interessantes:
* Ficheiros `.out`  contém o output principal do programa (incluindo os valores das propriedades termoquímicas).
* Ficheiros `.g98.out` contém os dados das vibrações no formato do software Gaussian 98, o qual pode ser usado por vários pacotes de visualização, tais como o molden ou o avogadro.
* Ficheiros `.charges` com a carga de Muliken de cada átomo no sistema
* Ficheiros `.wbo` com a ordem de ligação de Widberg entre os átomos do sistema.
* Fihcheiros `.vibspectrum` com o sumário do espectro de Infravermelho e Raman (números de onda, intensidades e aplicação das regras de selecção).

No nosso caso, vamos usar uma função definida no início do _notebook_ para animar o modo de vibração correspondente à coordenada de reacção, a partir do ficheiro `g98.out`). Olhando para o conteúdo do ficheiro `etilamina-mebr-dcm-ts-freq.g98.out`, podemos ver que o modo vibracional com frequência assciada negativa (imaginária) é o modo número 1. A função ```g98_animate_mode()``` converte a informação vibracional num ficheiro XYZ com várias _frames_ com a animação do modo vibracional, o qual pode ser observado no VMD.

O ficheiro ```etilamina-mebr-dcm-ts-freq.out``` contém também os valores absolutos de energia, entalpia e função de Gibbs para o estado de transição.

In [ ]:
g98_animate_mode('etilamina-mebr-dcm-ts-freq.g98.out',1,'etilamina-mebr-dcm-ts-animate.xyz')

Por fim, extraimos os pontos 15 e 5 da animação da coordenada de reacção, e usamos como geometria inicial para optimizar as geoemtrias de reagentes e produtos. **É importante verificar na animação da coordenada de reacção qual destes pontos representa uma aproximação aos produtos e qual deles representa uma apriximação aos produtos!** O xtb permite fazer o cálculo das frequências automaticamente após a optimização da geometria (`--ohess`). Mais uma vez, atente à nomenclatura dos ficheiros e anote os valores absolutos de energia, entalpia e função de Gibbs para produtos e reagentes.

In [ ]:
extract_geo('etilamina-mebr-dcm-ts-animate.xyz',15,'etilamina-mebr-dcm-react-newguess.xyz')
extract_geo('etilamina-mebr-dcm-ts-animate.xyz',5,'etilamina-mebr-dcm-prod-newguess.xyz')
print(run_xtb("etilamina-mebr-dcm-react-newguess.xyz --ohess --namespace etilamina-mebr-dcm-react-ohess --alpb CH2Cl2"), file=open('etilamina-mebr-dcm-react-ohess.out','w'))
print(run_xtb("etilamina-mebr-dcm-prod-newguess.xyz --ohess --namespace etilamina-mebr-dcm-prod-ohess --alpb CH2Cl2"), file=open('etilamina-mebr-dcm-prod-ohess.out','w'))

Aplique o procedimento acima descrito para estudar as outras três reacções da opção escolhida:
* Opção I: deverá modificar o átomo de halogénio na geometria inicial, e substituir mebr por meX (X sendo o halogénio) nos nomes dos ficheiros.
* Opção II: deverá substituir o solvente em `--alpb ch2cl2` por `hexane`, `acetonitrlie`, ou `water`. Nos nomes dos ficheiros, deverá substituir dcm por uma abreviatura adequada para o nome do solvente (ex: hex, actn e water).

Em ambos os casos, recomenda-se que faça copiar e colar do conteúdo das células de código da parte tutorial para novas células de código (abaixo) e modifique conforme necessário.

Em ambos os casos, deverá fazer uma tabela (recomenda-se uma folha de cálculo) com os valores de $\Delta E$, $\Delta H$ e,  $\Delta G$ para cada reacção, assim como as correspondentes energias de activação e variação da função de Gibbs na activação.

No final da aula, deverá enviar ao docente os ficheiros xtbopt.xyz do cálculo `ohess` de todos os reagentes e produtos, assim como os ficheiros xtbhess.xyz de todos os estados de transição.